# Project 1

Max Schrader

AEM 591

#### Resources

https://towardsdatascience.com/on-simulating-non-linear-dynamic-systems-with-python-or-how-to-gain-insights-without-using-ml-353eebf8dcc3

## Pre-Processing

In [32]:
import os
from dataclasses import dataclass
import math
from typing import Union

import pint
import numpy as np
# import control

In [2]:
ureg = pint.UnitRegistry()

### Constants

In [3]:
@dataclass
class Elevator:

    I_m = 700 * ureg.kilogram * ureg.meter ** 2
    K = 100 * ureg.newton * ureg.meter / ureg.ampere
    L = 0.4 * ureg.H
    m = 500 * ureg.kg
    g = 1 * ureg.gravity
    W = 300 * ureg.kilogram # unclear on what the units of this are
    r_0 = 3 * ureg.meter
    delta_c = 0.05 * ureg.meter
    R_0 = 5 * ureg.ohm
    delta_r = 5 * ureg.ohm
    t_r = 3 * ureg.second

### Given Equations

In [4]:
# def F_c_R_c(t: ureg.second, )
# Elevator.K * ureg.ampere + (Elevator.I_m / ureg.second ** 2 )/ ureg.meter
K = 1 * ureg.volt * ureg.second 
J = 1 * ureg.kilogram * ureg.meter ** 2
theta = (ureg.radian / ureg.second**2)
K * ureg.amp + J * theta

2.0 <Unit('ampere * second * volt')>

## Problem 1

Derive a state-space formulation using $x$, $\dot{x}$, $i$, and $r$ as the states, $v(t)$ as the input, and $x$ and $\dot{x}$ as the outputs.

$$\mathbf{\dot{s}} (t)=\mathbf {A} (t)\mathbf {s} (t)+\mathbf {B} (t)\mathbf {u} (t)$$
$$\mathbf{y} (t)=\mathbf {C} (t)\mathbf {s} (t)+\mathbf {D} (t)\mathbf {u} (t)$$

$$ \mathbf{s} = \begin{bmatrix}
x \\
\dot{x} \\
i \\
r 
\end{bmatrix},\; \mathbf{\dot{s}} = \begin{bmatrix}
\dot{x} \\
\ddot{x} \\
\dot{i} \\
\dot{r} 
\end{bmatrix}$$

$$\mathbf{u} (t) = \begin{bmatrix} 
v(t) \\
1
\end{bmatrix}
$$

$$
\mathbf{y} (t) = \begin{bmatrix} 
x \\
\dot{x}
\end{bmatrix}
$$


$$\begin{align*}
    \dot{x} &= 0 x + 1 \dot{x} + 0 i + 0 r + 0 v(t) \\
    \ddot{x} &= 0 x + 0 \dot{x} + i \frac{K}{r m - \frac{I_m}{r}} + 0 v(t) + \frac{w - m}{m - \frac{I_m}{r^2}} g\\
    \dot{i} &= 0 x - \frac{K}{L r(t)} \dot{x} - \frac{R(t)}{L} i + 0 r + \frac{1}{L} v(t) \\
    \dot{r} &= 0x + \frac{\delta_c}{2 \pi r} \dot x + 0 i + 0 r + 0 v(t)
\end{align*}$$

Where, 

$$G(t) = \frac{I_m}{r(t)} - r(t) m$$


In matrix form,

$$A(t) = \begin{bmatrix}
0 & 1 & 0 & 0 \\
0 & 0 & \frac{K}{r m - \frac{I_m}{r}} & 0 \\
0 & \frac{K}{L r(t)} & \frac{R(t)}{L} & 0 \\
0 & \frac{\delta_c}{2 \pi r} & 0 & 0 
\end{bmatrix}$$

$$B(t) = \begin{bmatrix}
0 & 0 \\
0 & \frac{w - m}{m - \frac{I_m}{r^2}} g \\
\frac{1}{L} & 0\\
0 & 0 
\end{bmatrix}$$


$$C(t) = \begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\ 
\end{bmatrix}$$

$$D(t) = \begin{bmatrix}
0 & 0 \\
0 & 0 \\ 
\end{bmatrix}$$

All together,

$$
\begin{bmatrix}
\dot{x} \\
\ddot{x} \\
\dot{i} \\
\dot{r} 
\end{bmatrix} = 
\begin{bmatrix}
0 & 1 & 0 & 0 \\
0 & 0 & \frac{K}{r m - \frac{I_m}{r}} & 0 \\
0 & \frac{K}{L r(t)} & \frac{R(t)}{L} & 0 \\
0 & \frac{\delta_c}{2 \pi r} & 0 & 0 
\end{bmatrix}
\begin{bmatrix}
x \\
\dot{x} \\
i \\
r 
\end{bmatrix} + 
\begin{bmatrix}
0 & 0 \\
0 & \frac{w - m}{m - \frac{I_m}{r^2}} g \\
\frac{1}{L} & 0\\
0 & 0 
\end{bmatrix}
\begin{bmatrix} 
v(t) \\
1
\end{bmatrix}
$$


$$\begin{bmatrix}
x \\
\dot{x}
\end{bmatrix} = 
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
x \\
\dot{x} \\
i \\
r 
\end{bmatrix} + 
\begin{bmatrix}
0 & 0 \\
0 & 0
\end{bmatrix}
\begin{bmatrix} 
v(t) \\
1
\end{bmatrix}$$

Other maybe useful equations?

$$ \dot{r} (t) =\frac{ \delta_c \dot{x} (t)}{2 \pi r} = \frac{dr}{dt} = \frac{ \delta_c \frac{dx}{dt} (t)}{2 \pi r}$$
$$ 2 \pi r dr = \delta_c dx$$
$$ \int 2 \pi r dr = \int \delta_c dx$$
$$ \pi r^2 + r_0 = \delta_c x + x_0$$
$$ x = \frac{\pi r^2 + r_0}{\delta_c}$$

### Nonlinear representation

$$\mathbf {\dot {x}} (t)=\mathbf {f} (t,x(t),u(t))$$

In [5]:
# g = lambda x:  
class ElevatorNonLinear(Elevator):
    def __init__(self, ):
        pass
    
    @property
    def v_t(self, ) -> pint.unit:
        return 0 * ureg.volt
    
    def r_t(self, x_t: np.array) -> pint.unit: 
        return x_t[-1]
    
    def _g_t(self, r_t: float) -> pint.unit:
        return self.I_m / r_t - r_t * self.m
    
    def _R_t(self, t) -> pint.unit:
        return self.R_0 + self.delta_r * (1 - math.exp(-t / (self.t_r)))
    
#     def _g_x(self, x):
        
    def f(self, ) -> object:
        def _f(x_t: np.array, t: float) -> np.array:
            # only calc G(t) once
            g_t = self._g_t(x_t[-1])
            R_t = self._R_t(t)
            r_t = self.r_t(x_t)
            
            x_dot = np.zeros(4)
            x_dot[0] = x_t[1]
            x_dot[1] = self.K / (r_t * self.m  - self.I_m / r_t) * x_t[2] + (self.w - self.m) / (self.m - (self.I_m / r_t ** 2)) * self.g
            x_dot[2] = self.K * x_t[1] / (self.L * r_t) +  R_t / self.L * x_t[2] + self.v_t / self.L
            x_dot[3] = self.delta_r * x_t[1] / (2 * math.pi * r_t)
            return x_dot
        return _f
    
    def out(self, ) -> object:
        def _out(x_t: np.array, t: float) -> np.array:
            out_dot = np.zeros(2)
            out_dot[0] = x_t[0]
            out_dot[1] = x_t[1]
            return out_dot
        return _out

## Problem 2

Linearize the state-space system about equilibrium at $x = 0$, $15$, and $30$ m for a value of $R = 10 \Omega$ which occurs as $ t \rightarrow \infty$. Analyze the controllability, observability, and stability.

In [113]:
class ElevatorLinear(ElevatorNonLinear):
    def __init__(self, x: float):
        self.x = x * ureg.meter
    
    def r_t(self, *args, **kwargs) -> pint.unit:
        return (self.r_0 ** 2 - self.delta_c * self.x / math.pi) ** (1/2)  
    
    def _R_t(self, *args, **kwargs) -> pint.unit:
        return 10 * ureg.ohm
    
    @property
    def A(self, ) -> np.array:
        
        r_t = self.r_t()
        
        return np.array([
            [0, 1, 0, 0],
            [0, 0, (self.K / (r_t * self.m  - self.I_m / r_t)).magnitude, 0],
            [0, (self.K / (self.L * r_t)).magnitude, (self._R_t() / self.L).magnitude, 0],
            [0, (self.delta_r / (2 * math.pi * r_t)).magnitude, 0, 0]
        ])

In [114]:
line_0 = ElevatorLinear(0)

In [115]:
r_t = line_0.r_t()
r_t.magnitude

3.0

In [107]:
v = (line_0.K / (r_t * line_0.m  - line_0.I_m / r_t)).magnitude

In [112]:
# (line_0._R_t() / line_0.L).magnitude
# (line_0.K / (line_0.L * r_t)).magnitude
(line_0.delta_r / (2 * math.pi * r_t)).magnitude

0.2652582384864922

In [116]:
line_0.A

array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.89473684e-02, 0.00000000e+00],
       [0.00000000e+00, 8.33333333e+01, 2.50000000e+01, 0.00000000e+00],
       [0.00000000e+00, 2.65258238e-01, 0.00000000e+00, 0.00000000e+00]])

## Problem 3

Simulate the nonlinear system with doublet commands on $\dot{v}$, both up and down, for initial conditions close and far from the linearized states, but including $t$. Use input values which keep the elevator velocity reasonable

## Problem 4

Simulate the linearized system with doublet commands on $dot{v}$, both up and down, for initial conditions close to the linearized states. Use input values which keep the elevator velocity reasonable

## Problem 5

Write a few paragraphs summarizing the results of these simulations and any analysis. Make sure to compare the linear simulations to the nonlinear.